In [1]:
# Description

# TODO
# https://stackoverflow.com/questions/58257251/how-to-check-if-a-file-exists-in-another-folder

# Requirements
import os
import geopandas as gpd
import pandas as pd
import pathlib

# Parameters
data_foldername = 'data'
data_directory = pathlib.Path('.').absolute() / data_foldername
data_directory = str(data_directory)
data_directory = data_directory + '\\'

gpx_foldername = 'gpxfiles'
gpx_directory = pathlib.Path('.').absolute() / gpx_foldername

#new_gpx_foldername = 'new_gpxfiles'
#gpx_directory = pathlib.Path('.').absolute() / new_gpx_foldername 

#processed_gpx_foldername = 'processed_gpxfiles'
#processed_gpx_directory = pathlib.Path('.').absolute() / processed_gpx_foldername 

# List of GeoPackage files to read
gpkg_files = os.listdir(data_directory)

In [2]:
## Proces for admin regions 0 (country-level)

# Initialize an empty list to store GeoDataFrames
gdf_list = []

# Loop through each GeoPackage file and read the data
for gpkg_file in gpkg_files:
    # Read the GeoPackage file
    gdf_file = gpd.read_file(os.path.join(data_directory,  gpkg_file), layer='ADM_ADM_0')

    gdf_list.append(gdf_file)

gdf_adm0 = pd.concat(gdf_list, ignore_index=True)

# Reset the index of the merged GeoDataFrame
gdf_adm0 = gdf_adm0.reset_index(drop=True)

#gdf_adm0.explore()

# Initialize an empty list to store GeoDataFrames
gdf_adminregions_per_gpx = []

# Proces per gpx-file
for filename in os.listdir(gpx_directory):
    if filename.endswith('.gpx'):
        gpxfile = os.path.join(gpx_directory, filename)
        gpx = gpd.read_file(gpxfile, layer='tracks')
        gdf_gpxline = gpx[gpx['name'].str.contains("Running")]
        
        # Perform the intersection
        gdf_join = gpd.sjoin(left_df=gdf_adm0, right_df=gdf_gpxline,  how="inner", predicate="intersects")
        #print(gdf_join)
        
        #Append data to temp gdf
        gdf_adminregions_per_gpx.append(gdf_join)

# Create gdf with information from all gpx-files
gdf_runs_adm0 = gpd.GeoDataFrame( pd.concat(gdf_adminregions_per_gpx, ignore_index=True) )

# Select columns
#col_list = ['GID_0', 'COUNTRY', 'geometry']
col_list = ['COUNTRY', 'geometry']
gdf_runs_adm0 = gdf_runs_adm0[col_list]
gdf_runs_adm0.columns = gdf_runs_adm0.columns.str.lower()


gdf_runs_adm0 = gdf_runs_adm0.drop_duplicates(["geometry"])

# Write geojsonfile
if(os.path.isfile("adm_0.geojson")):
    os.remove("adm_0.geojson")
    print("File Deleted successfully")
else:
    print("File does not exist")

gdf_runs_adm0.to_file("adm_0.geojson", driver='GeoJSON')

#gdf_runs_adm0.explore()
#gdf_runs_adm0.head()

File Deleted successfully


In [3]:
## Proces for admin regions 1 (province-level)

# Initialize an empty list to store GeoDataFrames
gdf_list = []

# Loop through each GeoPackage file and read the data
for gpkg_file in gpkg_files:
    # Read the GeoPackage file
    gdf_file = gpd.read_file(os.path.join(data_directory,  gpkg_file), layer='ADM_ADM_1')
    gdf_list.append(gdf_file)

gdf_adm1 = pd.concat(gdf_list, ignore_index=True)

gdf_adm1 = gdf_adm1[~gdf_adm1['ENGTYPE_1'].str.contains("Water body")]

# Reset the index of the merged GeoDataFrame
gdf_adm1 = gdf_adm1.reset_index(drop=True)

# Initialize an empty list to store GeoDataFrames
gdf_adminregions_per_gpx = []

# Proces per gpx-file
for filename in os.listdir(gpx_directory):
    if filename.endswith('.gpx'):
        gpxfile = os.path.join(gpx_directory, filename)
        gpx = gpd.read_file(gpxfile, layer='tracks')
        gdf_gpxline = gpx[gpx['name'].str.contains("Running")]

        # Perform the intersection
        gdf_join = gpd.sjoin(left_df=gdf_adm1, right_df=gdf_gpxline,  how="inner", predicate="intersects")
        #print(gdf_join)
        
        #Append data to temp gdf
        gdf_adminregions_per_gpx.append(gdf_join)

# Create gdf with information from all gpx-files
gdf_runs_adm1 = gpd.GeoDataFrame( pd.concat(gdf_adminregions_per_gpx, ignore_index=True) )

# Select columns
#col_list = ['GID_0', 'GID_1', 'COUNTRY', 'NAME_1', 'TYPE_1', 'geometry']
col_list = ['COUNTRY', 'NAME_1', 'TYPE_1', 'geometry']
gdf_runs_adm1 = gdf_runs_adm1[col_list]
gdf_runs_adm1 = gdf_runs_adm1.rename(columns={"NAME_1": "NAME"})
gdf_runs_adm1 = gdf_runs_adm1.rename(columns={"TYPE_1": "TYPE"})
gdf_runs_adm1.columns = gdf_runs_adm1.columns.str.lower()

gdf_runs_adm1 = gdf_runs_adm1.drop_duplicates(["geometry"])

# Write geojsonfile
if(os.path.isfile("adm_1.geojson")):
    os.remove("adm_1.geojson")
    print("File Deleted successfully")
else:
    print("File does not exist")

gdf_runs_adm1.to_file("adm_1.geojson", driver='GeoJSON')

#gdf_runs_adm1.explore()
#gdf_runs_adm1.head()

File Deleted successfully


In [4]:
## Proces for admin regions 2 (municipality-level)

# Initialize an empty list to store GeoDataFrames
gdf_list = []

# Loop through each GeoPackage file and read the data
for gpkg_file in gpkg_files:
    # Read the GeoPackage file
    gdf_file = gpd.read_file(os.path.join(data_directory,  gpkg_file), layer='ADM_ADM_2')
    gdf_list.append(gdf_file)

gdf_adm2 = pd.concat(gdf_list, ignore_index=True)

gdf_adm2 = gdf_adm2[~gdf_adm2['ENGTYPE_2'].str.contains("Water body")]

# Reset the index of the merged GeoDataFrame
gdf_adm2 = gdf_adm2.reset_index(drop=True)

# Initialize an empty list to store GeoDataFrames
gdf_adminregions_per_gpx = []

# Proces per gpx-file
for filename in os.listdir(gpx_directory):
    if filename.endswith('.gpx'):
        gpxfile = os.path.join(gpx_directory, filename)
        gpx = gpd.read_file(gpxfile, layer='tracks')
        gdf_gpxline = gpx[gpx['name'].str.contains("Running")]
        
        # Perform the intersection
        gdf_join = gpd.sjoin(left_df=gdf_adm2, right_df=gdf_gpxline,  how="inner", predicate="intersects")
        #print(gdf_join)
        
        #Append data to temp gdf
        gdf_adminregions_per_gpx.append(gdf_join)

# Create gdf with information from all gpx-files
gdf_runs_adm2 = gpd.GeoDataFrame( pd.concat(gdf_adminregions_per_gpx, ignore_index=True) )

# Select columns
#col_list = ['GID_0', 'GID_1', 'GID_2', 'COUNTRY', 'NAME_2', 'TYPE_2', 'geometry']
col_list = ['COUNTRY', 'NAME_2', 'TYPE_2',  'geometry']
gdf_runs_adm2 = gdf_runs_adm2[col_list]
gdf_runs_adm2 = gdf_runs_adm2.rename(columns={"NAME_2": "NAME"})
gdf_runs_adm2 = gdf_runs_adm2.rename(columns={"TYPE_2": "TYPE"})
gdf_runs_adm2.columns = gdf_runs_adm2.columns.str.lower()

gdf_runs_adm2 = gdf_runs_adm2.drop_duplicates(["geometry"])

# Write geojsonfile
if(os.path.isfile("adm_2.geojson")):
    os.remove("adm_2.geojson")
    print("File Deleted successfully")
else:
    print("File does not exist")

gdf_runs_adm2.to_file("adm_2.geojson", driver='GeoJSON')

#gdf_runs_adm2.explore()
#gdf_runs_adm2.head()

File Deleted successfully


In [5]:
# Useful statements

# import fiona
# layers = fiona.listlayers(file)

In [6]:
'''
# Move data from new_gpxfiles dir to processed_gpxfiles dir
# gather all files in source folder
allfiles = os.listdir(gpx_directory)
 
# iterate on all files to move them to destination folder
for f in allfiles:
    src_path = os.path.join(gpx_directory, f)
    dst_path = os.path.join(processed_gpx_directory, f)
    os.rename(src_path, dst_path)
    '''

'\n# Move data from new_gpxfiles dir to processed_gpxfiles dir\n# gather all files in source folder\nallfiles = os.listdir(gpx_directory)\n \n# iterate on all files to move them to destination folder\nfor f in allfiles:\n    src_path = os.path.join(gpx_directory, f)\n    dst_path = os.path.join(processed_gpx_directory, f)\n    os.rename(src_path, dst_path)\n    '